In [43]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as scs
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sqlite3

%config InlineBackend.figure_formats = ['svg']
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

# Load Data

In [45]:
df = pd.read_csv('../temperament.csv')
df = df.replace(' ', np.nan)
df.head()

,id,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
0,6085,1,0,1,1,13,6,3,2,2006,3,6,2006,8.20807665982204,0,2,2,1,1,12/16/1997,16,12,1997,2,8,3/2/2006,1,4,2,3,4,3,1,2,1,1,3,1,1,2,1,3,5,4,2,1,1,3,2.28571428571429,2.81818181818182,...,NaN,2.61030337923131,-.769877629744393,3.33333333333333,3.66666666666667,-1,-.0492703703703703,1.9726,.0375000000000001,.00327822962962963,-.61316,.51796,.0422285562962963,-.035672097037037,1,NaN,.0450666666666667,-.0644333333333333,.00417777777777778,-.0993333333333335,-.187376923076923,.0783142857142858,NaN,-.0546666666666667,.0644333333333333,.0614703703703703,-.00417777777777778,NaN,-.00977195555555556,.00679986444444444,.0106834586419753,-.000440894814814815,0,NaN,.00576915555555556,-.00679986444444444,-.00648717308641974,.000440894814814815,1.5808835261739,1.5808835261739,.059135533438049,0,0,0,-.157034430266608,0,.646883526106318,.930064032241653,.64178753842755,.939095987746353
1,11005,2,0,1,1,61,11,2,19,2007,2,27,2007,7.82477754962355,0,2,1,0,1,4/24/1999,24,4,1999,9,7,2/19/2007,1,2,1,2,3,3,1,2,1,1,1,1,2,2,2,2,3,2,2,2,1,1,1.76190476190476,1.90909090909091,...,NaN,NaN,NaN,1,1,-1,.116484615384615,-.2774,-.4625,-.0530540153846154,-1.11316,.0179599999999999,-.107848078461538,.00174004769230768,0,-.0335153846153846,-.0549846153846154,.0719769230769231,-.0166769230769231,.286777777777778,.120315384615385,-.445495238095238,.0335153846153846,.0453846153846154,-.0719769230769231,-.104284615384615,.0166769230769231,-.00940367213675215,-.00930767350427353,.0201951251282052,.0197182568376069,-.00467917401709402,0,.00940367213675215,.0127339145299146,-.0201951251282052,-.0292599456410257,.00467917401709402,NaN,NaN,NaN,0,0,0,NaN,0,NaN,NaN,NaN,NaN
2,11008,2,0,1,1,61,11,2,5,2007,2,19,2007,7.42778918548939,0,2,2,1,0,9/2/1999,2,9,1999,5,7,2/5/2007,1,1,2,2,1,1,1,1,1,1,1,1,2,1,2,2,1,1,2,1,1,1,1.28571428571429,1.18181818181818,...,NaN,NaN,NaN,1,1,-1,.132918181818182,-.2774,.5375,.0512651454545454,-.11316,1.01796,-.0128230854545455,.115353376363636,0,-.0398090909090909,-.0623272727272727,.0625363636363636,-.00269090909090909,-.179888888888889,-.2643,-.921685714285714,.0398090909090909,.0527272727272727,-.0625363636363636,-.120718181818182,.00269090909090909,.00740802949494948,.0185354383838384,-.0116373224242424,-.0395283904040404,.000500748282828282,0,-.00740802949494948,-.00981195959595958,.0116373224242424,.0224643123232323,-.000500748282828282,NaN,NaN,NaN,0,0,0,NaN,0,NaN,NaN,NaN,NaN
3,11009,2,0,1,1,61,11,2,5,2007,2,19,2007,8.29568788501027,0,2,2,1,1,10/20/1998,20,10,1998,3,8,2/5/2007,1,3,3,3,1,2,5,1,1,1,1,3,1,1,1,3,4,3,3,1,1,1,2.04761904761905,2.09090909090909,...,NaN,NaN,NaN,1,4.66666666666667,-1,.00462112676056337,-.2774,-.3375,.00633007183098593,-.98816,.14296,.0148015233802817,-.00214137971830986,0,-.0643929577464789,.120259154929577,-.0306774647887324,-.00909295774647888,.631222222222222,-.0335307692307691,-.159780952380952,.0643929577464789,-.129859154929577,.0306774647887324,.00757887323943662,.00909295774647888,-.04024702954616

# Select Rows for Cohort 1 with no missing data

In [46]:
aggdf = df[(df.cohort == 1) & (~df.mt1tovertagg.isnull()) & (~df.mt2tovertagg.isnull()) &
           (~df.mt1overtvic.isnull()) & (~df.mt1tovertvic.isnull()) & (~df.mt1picontrol.isnull())]
aggdf.head()

,id,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
0,6085,1,0,1,1,13,6,3,2,2006,3,6,2006,8.20807665982204,0,2,2,1,1,12/16/1997,16,12,1997,2,8,3/2/2006,1,4,2,3,4,3,1,2,1,1,3,1,1,2,1,3,5,4,2,1,1,3,2.28571428571429,2.81818181818182,...,NaN,2.61030337923131,-.769877629744393,3.33333333333333,3.66666666666667,-1,-.0492703703703703,1.9726,.0375000000000001,.00327822962962963,-.61316,.51796,.0422285562962963,-.035672097037037,1,NaN,.0450666666666667,-.0644333333333333,.00417777777777778,-.0993333333333335,-.187376923076923,.0783142857142858,NaN,-.0546666666666667,.0644333333333333,.0614703703703703,-.00417777777777778,NaN,-.00977195555555556,.00679986444444444,.0106834586419753,-.000440894814814815,0,NaN,.00576915555555556,-.00679986444444444,-.00648717308641974,.000440894814814815,1.5808835261739,1.5808835261739,.059135533438049,0,0,0,-.157034430266608,0,.646883526106318,.930064032241653,.64178753842755,.939095987746353
174,1002,1,0,1,1,17,1,1,11,2006,1,11,2006,7.03901437371663,0,2,2,1,0,12/28/1998,28,12,1998,0,7,1/11/2006,1,5,1,5,1,5,5,5,5,5,5,1,1,1,1,1,5,3,5,5,5,5,3.57142857142857,4.81818181818182,...,-1.13589152399609,1.85298875543702,.635096408372729,1,1,-1,-.0881857142857143,-.2774,-.0874999999999999,.0186038142857143,-.73816,.39296,.0795631028571429,-.0423554742857143,1,-.0417571428571429,-.0824571428571429,-.0118142857142857,-.0400285714285714,-.921555555555556,-.2643,1.36402857142857,.0417571428571429,.0728571428571429,.0118142857142857,.100385714285714,.0400285714285714,.0387404212698413,.0856775523809524,.0109607692063492,.0916298563492063,.0371367295238095,0,-.0387404212698413,-.067593619047619,-.0109607692063492,-.0931334041269841,-.0371367295238095,2.49820787542738,2.49820787542738,-.220192272420725,0,0,0,-2.30223734653275,0,2.90952219663325,-.393039083253536,2.88383271860357,-.385624843176193
175,1004,1,0,1,1,17,1,1,11,2006,1,11,2006,7.46885694729637,0,2,6,0,0,7/24/1998,24,7,1998,5,7,1/11/2006,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,3,3,1,1,3,3,1.47619047619048,1.90909090909091,...,-1.6196768814573,.999121866575311,.910435486162086,1,2.66666666666667,-1,.0893352941176471,-.2774,.0375000000000001,-.00331940000000001,-.61316,.51796,-.0427588929411765,.0361200929411765,1,-.0165470588235294,-.0866588235294118,-.0128647058823529,.00934117647058823,-.274333333333333,-.2643,-.731209523809524,.0165470588235294,.0770588235294118,.0128647058823529,-.0771352941176471,-.00934117647058823,.00464200156862745,.0339558156862745,.00360897882352941,-.0350045460784313,-.00262051137254901,0,-.00464200156862745,-.021617568627451,-.00360897882352941,.0216390211764706,.00262051137254901,-.7629487502832,-.7629487502832,-.0290485380042753,0,0,0,.209302273827691,0,-.38158859686047,-.393039083253536,-.377323907107007,-.385624843176193
176,1006,1,0,1,1,17,1,1,11,2006,1,11,2006,7.69062286105407,0,2,2,1,1,5/4/1998,4,5,1998,8,7,1/11/2006,1,5,3,5,5,5,5,5,5,4,5,5,4,4,4,4,4,4,5,5,4,5,4.52380952380952,4.72727272727273,...,2.45286433332974,-.43974919913004,-.332146181237503,1,1,-1,.0213702